> Demo modified from https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/0762b167-402a-4362-9702-7d559f0e73bb/notebook/iris_builder-3c25c10bc29d4c9493e48eaced475d03

In [ ]:
import os
import numpy as np
from collections import OrderedDict

from pydrake.geometry import StartMeshcat, QueryObject, Role
from pydrake.multibody.parsing import PackageMap, Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.tree import Body
from pydrake.systems.framework import DiagramBuilder, LeafSystem
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.inverse_kinematics import InverseKinematics
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.all import LoadIrisRegionsYamlFile, HPolyhedron, Sphere, Rgba

from iiwa_shelve_and_bins import run, get_cfg, get_gcs, gcs_convex_restriction

In [ ]:
def LoadRobot(plant: MultibodyPlant) -> Body:
    """Setup your plant, and return the body corresponding to your
    end-effector."""
    parser = Parser(plant)

    # We'll use some tables, shelves, and bins from a remote resource.
    parser.package_map().AddRemote(
        package_name="gcs",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/mpetersen94/gcs/archive/refs/tags/arxiv_paper_version.tar.gz"
            ],
            sha256=("6dd5e841c8228561b6d622f592359c36517cd3c3d5e1d3e04df74b2f5435680c"),
            strip_prefix="gcs-arxiv_paper_version",
        ),
    )

    model_directives = """    
directives:

# Add iiwa
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.3]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.8]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1]
        iiwa_joint_7: [1.57]

- add_weld:
    parent: world
    child: iiwa::base

# Add schunk
- add_model:
    name: wsg
    file: package://drake_models/wsg_50_description/sdf/schunk_wsg_50_welded_fingers.sdf

- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.114]
      rotation: !Rpy { deg: [90.0, 0.0, 0.0 ]}

# Add shelves
- add_model:
    name: shelves
    file: package://gcs/models/shelves/shelves.sdf

- add_weld:
    parent: world
    child: shelves::shelves_body
    X_PC:
      translation: [0.85, 0, 0.4]

# Add Bins
- add_model:
    name: binR
    file: package://gcs/models/bin/bin.sdf

- add_weld:
    parent: world
    child: binR::bin_base
    X_PC:
      translation: [0, -0.6, 0]
      rotation: !Rpy { deg: [0.0, 0.0, 90.0 ]}

- add_model:
    name: binL
    file: package://gcs/models/bin/bin.sdf

- add_weld:
    parent: world
    child: binL::bin_base
    X_PC:
      translation: [0, 0.6, 0]
      rotation: !Rpy { deg: [0.0, 0.0, 90.0 ]}

# Add table
- add_model:
    name: table
    file: package://gcs/models/table/table_wide.sdf

- add_weld:
    parent: world
    child: table::table_body
    X_PC:
      translation: [0.4, 0.0, 0.0]
      rotation: !Rpy { deg: [0., 0., 00]}
"""

    parser.AddModelsFromString(model_directives, ".dmd.yaml")
    gripper = plant.GetModelInstanceByName("wsg")
    end_effector_body = plant.GetBodyByName("body", gripper)
    return end_effector_body


def DrawRobot(meshcat, query_object: QueryObject, meshcat_prefix: str, draw_world: bool = True):
    rgba = Rgba(0.7, 0.7, 0.7, 0.3)
    role = Role.kProximity
    # This is a minimal replication of the work done in MeshcatVisualizer.
    inspector = query_object.inspector()
    for frame_id in inspector.GetAllFrameIds():
        if frame_id == inspector.world_frame_id():
            if not draw_world:
                continue
            frame_path = meshcat_prefix
        else:
            frame_path = f"{meshcat_prefix}/{inspector.GetName(frame_id)}"
        frame_path.replace("::", "/")
        frame_has_any_geometry = False
        for geom_id in inspector.GetGeometries(frame_id, role):
            # shadows/0/iiwa
            if frame_path.split("/")[2][:4] == "iiwa" or frame_path.split("/")[2][:3] == "wsg":
                path = f"{frame_path}/{geom_id.get_value()}"
                path.replace("::", "/")
                meshcat.SetObject(path, inspector.GetShape(geom_id), rgba)
                meshcat.SetTransform(path, inspector.GetPoseInFrame(geom_id))
                frame_has_any_geometry = True

        if frame_has_any_geometry:
            X_WF = query_object.GetPoseInWorld(frame_id)
            meshcat.SetTransform(frame_path, X_WF)

# Sometimes it's useful to use inverse kinematics to find the seeds. You might
# need to adapt this to your robot. This helper takes an end-effector frame, E,
# and a desired pose for that frame in the world coordinates, X_WE.
def MyInverseKinematics(X_WE, plant=None, context=None):
    if not plant:
        plant = MultibodyPlant(0.0)
        LoadRobot(plant)
        plant.Finalize()
    if not context:
        context = plant.CreateDefaultContext()
    # E = ee_body.body_frame()
    E = plant.GetBodyByName("body").body_frame()

    ik = InverseKinematics(plant, context)

    ik.AddPositionConstraint(
        E, [0, 0, 0], plant.world_frame(), X_WE.translation(), X_WE.translation()
    )

    ik.AddOrientationConstraint(
        E, RotationMatrix(), plant.world_frame(), X_WE.rotation(), 0.001
    )

    prog = ik.get_mutable_prog()
    q = ik.q()

    q0 = plant.GetPositions(context)
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print("IK failed")
        return None
    plant.SetPositions(context, result.GetSolution(q))
    return result.GetSolution(q)


def get_default_position():
    plant = MultibodyPlant(0.0)
    LoadRobot(plant)
    plant.Finalize()
    context = plant.CreateDefaultContext()
    return plant.GetPositions(context)


def PublishPositionTrajectory(
    meshcat, trajectory, targets, root_context, scene_graph, plant:MultibodyPlant, visualizer, time_step=1.0 / 50.0
):
    """
    Args:
        trajectory: A Trajectory instance.
    """
    plant_context = plant.GetMyContextFromRoot(root_context)
    visualizer_context = visualizer.GetMyContextFromRoot(root_context)
    scene_graph_context = scene_graph.GetMyContextFromRoot(root_context)

    for ii, q in enumerate(targets.values()):
        plant.SetPositions(plant_context, q)
        # query = scene_graph.get_query_output_port().Eval(scene_graph_context)
        # DrawRobot(meshcat, query, f"shadows/{ii}", False)
        
        gripper = plant.GetModelInstanceByName("wsg")
        end_effector_body = plant.GetBodyByName("body", gripper)
        X_WE = plant.CalcRelativeTransform(
            plant_context,
            frame_A = plant.world_frame(),
            frame_B = end_effector_body.body_frame(),
        )

        meshcat.SetObject("targets/" + str(ii), Sphere(0.04), Rgba(1.0, 0.0, 0.0, 0.5))
        tf = RigidTransform(RollPitchYaw(0, 0, 0), X_WE.translation())
        meshcat.SetTransform("targets/" + str(ii), tf)

    visualizer.StartRecording(False)

    for ii, t in enumerate(np.append(
        np.arange(trajectory.start_time(), trajectory.end_time(), time_step),
        trajectory.end_time(),
    )):
        root_context.SetTime(t)
        state = trajectory.value(t)
        plant.SetPositions(plant_context, state)
        visualizer.ForcedPublish(visualizer_context)

        gripper = plant.GetModelInstanceByName("wsg")
        end_effector_body = plant.GetBodyByName("body", gripper)
        X_WE = plant.CalcRelativeTransform(
            plant_context,
            frame_A = plant.world_frame(),
            frame_B = end_effector_body.body_frame(),
        )

        meshcat.SetObject("traj/sample_" + str(ii), Sphere(0.01), Rgba(0.0, 0.0, 0.0, 0.5))
        tf = RigidTransform(RollPitchYaw(0, 0, 0), X_WE.translation())
        meshcat.SetTransform("traj/sample_" + str(ii), tf)

    visualizer.StopRecording()
    visualizer.PublishRecording()
    

In [ ]:
meshcat = StartMeshcat()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
end_effector_body = LoadRobot(plant)
plant.Finalize()
AddDefaultVisualization(builder, meshcat)
diagram = builder.Build()

target_pts = OrderedDict()
target_pts["Home Position"] = get_default_position()
target_pts["Above Shelve"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, -np.pi, -np.pi / 2), [0.75, 0, 0.9])
)
target_pts["Top Rack"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, -np.pi, -np.pi / 2), [0.75, 0, 0.67])
)
target_pts["Middle Rack"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(0, -np.pi, -np.pi / 2), [0.75, 0, 0.41])
)
target_pts["Left Bin"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(np.pi / 2, np.pi, 0), [0.0, 0.6, 0.22])
)
target_pts["Right Bin"] = MyInverseKinematics(
    RigidTransform(RollPitchYaw(np.pi / 2, np.pi, np.pi), [0.0, -0.6, 0.22])
)
target_pts["Front to Shelve"] = np.array([0, 0.2, 0, -2.09, 0, -0.3, np.pi / 2])
target_pts["Left to Shelve"] = np.array([0.8, 0.7, 0, -1.6, 0, 0, np.pi / 2])
target_pts["Right to Shelve"] = np.array([-0.8, 0.7, 0, -1.6, 0, 0, np.pi / 2])

# pre-computed convex regions
iris_filename = "iiwa_shelve_and_bins_science_robotics.yaml"
iris_regions = dict()
assert os.path.isfile(iris_filename), f"File {iris_filename} not found."
iris_regions.update(LoadIrisRegionsYamlFile(iris_filename))

regions = list(iris_regions.values()) + [HPolyhedron.MakeBox(pt-0.01, pt+0.01) for pt in target_pts.values()]
targets = list(range(len(iris_regions), len(regions)))

vmin = plant.GetVelocityLowerLimits()
vmax = plant.GetVelocityUpperLimits()

In [ ]:
unfolded_path, gcs = run(vmin, vmax, regions, targets)

# unfolded_path = [9, 2, 8, 11, 8, 2, 1, 6, 7, 17, 7, 6, 14, 6, 7, 0, 10, 0, 4, 3, 13, 3, 4, 16, 4, 3, 1, 5, 12, 5, 15, 1, 5, 2, 9]
gcs = get_gcs(regions, vmin, vmax)
traj = gcs_convex_restriction(unfolded_path, gcs)

PublishPositionTrajectory(
    meshcat,
    traj,
    target_pts,
    diagram.CreateDefaultContext(),
    scene_graph,
    plant,
    diagram.GetSubsystemByName("meshcat_visualizer(illustration)"),)